In [0]:
# uploading of BindingDB_All_2019m3.tsv.zip

from google.colab import files
files.upload()

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp /content/drive/My\ Drive/BindingDB_All_2019m3.tsv.zip /content/

In [0]:
!unzip BindingDB_All_2019m3.tsv.zip

In [0]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('BindingDB_All.tsv', sep = '\t', error_bad_lines=False)

In [0]:
print(data.shape)
data = data.loc[data['Target Source Organism According to Curator or DataSource'] == 'Homo sapiens']
print(data.shape)

In [0]:
bad_val=[]
def convert_to_float(val, allowed_prefixes=('>', '<')):
    if isinstance(val, (int, float)):
        return float(val)
    
    val = str(val)
    if val[0] not in allowed_prefixes:
        #raise Exception(f'Conversion failed! Got "{val}"')
        bad_val.append(val)
        return 50000
        
    return float(val[1:])

def process_signs(val):
    val = str(val)
    if val[0]=='>':
        return 'greater'
    else:
        return 'else'

In [0]:
#allowed_prefixes=('>', '<') # will fail, since some items start with space
allowed_prefixes=('>', '<', ' ') # works nicely

for_select_ds = data

for_select_ds['IC50_signs'] = data['IC50 (nM)'].apply(process_signs)
for_select_ds['IC50_floats_data'] = data['IC50 (nM)'].apply(convert_to_float, allowed_prefixes=allowed_prefixes)
for_select_ds['IC50_nan'] = pd.isnull(for_select_ds['IC50_floats_data'])


for_select_ds['Kd_signs'] = data['Kd (nM)'].apply(process_signs)
for_select_ds['Kd_floats_data'] = data['Kd (nM)'].apply(convert_to_float, allowed_prefixes=allowed_prefixes)
for_select_ds['Kd_nan'] = pd.isnull(for_select_ds['Kd_floats_data'])


for_select_ds['EC50_signs'] = data['EC50 (nM)'].apply(process_signs)
for_select_ds['EC50_floats_data'] = data['EC50 (nM)'].apply(convert_to_float, allowed_prefixes=allowed_prefixes)
for_select_ds['EC50_nan'] = pd.isnull(for_select_ds['EC50_floats_data'])

for_select_ds.loc[(for_select_ds['IC50_nan'] == False) & (for_select_ds['IC50_floats_data'] < 100) & (for_select_ds['IC50_signs'] != 'greater'), 'IC50_res']=True
for_select_ds.loc[(for_select_ds['Kd_nan'] == False) & (for_select_ds['Kd_floats_data'] < 100) & (for_select_ds['Kd_signs'] != 'greater'), 'Kd_res']=True
for_select_ds.loc[(for_select_ds['EC50_nan'] == False) & (for_select_ds['EC50_floats_data'] < 100) & (for_select_ds['EC50_signs'] != 'greater'), 'EC50_res']=True


result=for_select_ds.loc[(for_select_ds['IC50_res'] == True)|((for_select_ds['Kd_res'] == True)&(for_select_ds['IC50_nan']==True))|((for_select_ds['Kd_nan'] == True)&(for_select_ds['IC50_nan']==True)&(for_select_ds['EC50_res'] == True))]
#result=for_select_ds.loc[(for_select_ds['IC50_res'] == True)|(for_select_ds['Kd_res'] == True)|(for_select_ds['EC50_res'] == True)]
#for_select_ds['IC50_res'] = ['True' if ((row['IC50_nan'] == False) & (row['IC50_floats_data'] < 100) & (row['IC50_signs'] != 'greater')) else 'False' for index, row in for_select_ds.iterrows()] 

In [0]:
result.shape

In [0]:
len(bad_val)

In [0]:
result2 = result.loc[pd.isnull(result['Ligand SMILES']) == False]
print(result2.shape)
with_pubchem_cid = result2.loc[pd.isnull(result2['PubChem CID']) == False]
print(with_pubchem_cid.shape)

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.6.7/site-packages/')
sys.path.append('/usr/local/lib/python3.6.7/site-packages/rdkit/')

In [0]:
% cd /usr/local/lib/python3.7/site-packages/

In [0]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols


In [0]:
def mol_weight(smile):
  try:
    return Descriptors.MolWt(Chem.MolFromSmiles(str(smile)))
  except Exception:
    return np.nan

In [0]:
with_pubchem_cid['mol_weight'] = with_pubchem_cid['Ligand SMILES'].apply(mol_weight)
print(with_pubchem_cid.shape)
with_mol_weight_smile =with_pubchem_cid.loc[(pd.isnull(with_pubchem_cid['mol_weight'])==False) & (with_pubchem_cid['mol_weight'] < 1001)]
print(with_mol_weight_smile.shape)

In [0]:
def process_protein_col(ds_to_start, var_name1,var_name2):
  prot_cond1 = ds_to_start.loc[pd.isnull(ds_to_start[var_name1]) == False]
  prot_cond2 = prot_cond1.loc[pd.isnull(prot_cond1[var_name2]) == False]
 # prot_cond2['prot_lens'] = prot_cond2[var_name1].apply(len)
  print('shape_before_len ',prot_cond2.shape)
  if prot_cond2.empty == False:
    prot_cond3 = prot_cond2.loc[((prot_cond2[var_name1].str.len() > 79) & (prot_cond2[var_name1].str.len() < 2049))]
    print('shape_after_len ',prot_cond3.shape)
    return prot_cond3
  else:
    return prot_cond2
  

'''result_only_1_chain = with_mol_weight_smile.loc[with_mol_weight_smile['Number of Protein Chains in Target (>1 implies a multichain complex)'] == 1]
prot_cond1 = result_only_1_chain.loc[pd.isnull(result_only_1_chain['BindingDB Target Chain  Sequence']) == False]
prot_cond2 = prot_cond1.loc[pd.isnull(prot_cond1['UniProt (SwissProt) Primary ID of Target Chain']) == False]
prot_cond2['prot_lens'] = prot_cond2['BindingDB Target Chain  Sequence'].apply(len)
prot_cond3 = prot_cond2.loc[((prot_cond2['prot_lens'] > 79) & (prot_cond2['prot_lens'] < 4001))]'''

prot_cond0 = with_mol_weight_smile.loc[with_mol_weight_smile['Number of Protein Chains in Target (>1 implies a multichain complex)'] == 1]
prot_1_chain = process_protein_col(ds_to_start = prot_cond0, var_name1 = 'BindingDB Target Chain  Sequence',var_name2 = 'UniProt (SwissProt) Primary ID of Target Chain')
print('prot_1_chain',prot_1_chain.shape)


with open('/content/proteins_and_ligands', 'a') as prot_file:
  prot_1_chain[['BindingDB Target Chain  Sequence','Ligand SMILES']].to_csv(prot_file, index=False, header=False, sep = '\t')  
  ids = list(set(list(prot_1_chain['BindingDB Target Chain  Sequence'])))
  for i in range(1, 13):
    print('processing ',i)
    name1 = 'BindingDB Target Chain  Sequence.'+str(i)
    name2 = 'UniProt (SwissProt) Primary ID of Target Chain.'+str(i)
    prot_cond0 = with_mol_weight_smile.loc[with_mol_weight_smile['Number of Protein Chains in Target (>1 implies a multichain complex)'] > 1]
    temp_ds1 = process_protein_col(ds_to_start = prot_cond0, var_name1 = name1, var_name2 = name2)
    temp_ds1[[name1,'Ligand SMILES']].to_csv(prot_file, index=False, header=False, sep = '\t')
    print(temp_ds1.shape)
    ids.extend(list(set(list(temp_ds1[name1]))))
    prot_1_chain=prot_1_chain.append(temp_ds1)
    print('prot_1_chain',prot_1_chain.shape)
    


In [0]:
with open('/content/dataset_full',"w") as t:
  prot_1_chain.to_csv(t,sep='\t')

In [0]:
prot_1_chain.shape

In [0]:
print(len(set(ids)))

In [0]:
import re
import random 

def number_check(string):
    return re.search('\d', string)
  

  
len_list = 100  

unique_ids = list(set(ids))
print(len(unique_ids))
random.shuffle(unique_ids)
train_id_list = unique_ids[len_list:]
test_id_list = unique_ids[:len_list]

  


smis_test=[]
smis_train=[]
with open('/content/proteins_and_ligands') as for_read, open('/content/proteins_train', 'w') as prot_file_train, open('/content/proteins_test', 'w') as prot_file_test:
  for line in for_read:
    line_splitted = line.strip().split("\t")
    
    if number_check(line_splitted[0])==None:
      if line_splitted[0] in train_id_list:
        prot_file_train.write(line_splitted[0].upper()+'\n')
        smis_train.append(line_splitted[1])
      else:
        prot_file_test.write(line_splitted[0].upper()+'\n')
        smis_test.append(line_splitted[1])
        
    
    

  

cans_train = [Chem.MolToSmiles(Chem.MolFromSmiles(smi),True) for smi in smis_train]
cans_test = [Chem.MolToSmiles(Chem.MolFromSmiles(smi),True) for smi in smis_test]
with open('/content/ligands_train',"w") as canonical_smi_train:
  for smile in cans_train:
    canonical_smi_train.write(smile+'\n')
    
    
with open('/content/ligands_test', 'w') as canonical_smi_test:
  for smile in cans_test:
    canonical_smi_test.write(smile+'\n')
        

In [0]:
import os



def create_dictionary(uniqe_symbols, file_name):
    with open(
            dir+file_name,
            "r") as drug_protein_connection:
        for line in drug_protein_connection:
            for symbol in line.strip():
                if symbol not in uniqe_symbols:
                    uniqe_symbols.append(symbol)
    return uniqe_symbols


def write_dict_to_file(uniqe_symbols,file_name):
    output_file = open(
            dir+file_name,
            "w")
    for symbol in uniqe_symbols:
        output_file.write("'"+symbol+"'" + "\n")
    return


def create_file_with_space_separated_string_of_tokens(file_name):
    if os.path.exists(dir + file_name+".space_sep_seq"):
        os.remove(dir + file_name+".space_sep_seq")
    output_file = open(
        dir + file_name+".space_sep_seq",
        "a")
    with open(dir + file_name,"r") as drug_protein_connection:
        for line in drug_protein_connection:
                    output_file.write(" ".join(line.strip())+"\n")
    return

In [0]:
dir = "/content/"
uniqe_symbols = []
dict_list1 = create_dictionary(uniqe_symbols, "proteins_and_ligands")
write_dict_to_file(uniqe_symbols=dict_list1,file_name="vocab_from_prot_and_lig")




uniqe_symbols = []
dict_list1 = create_dictionary(uniqe_symbols, "ligands_train")
dict_list2 = create_dictionary(dict_list1, "ligands_test")
dict_list3 = create_dictionary(dict_list2, "proteins_train")
dict_list4 = create_dictionary(dict_list3, "proteins_test")
write_dict_to_file(uniqe_symbols=dict_list4,file_name="vocab")

create_file_with_space_separated_string_of_tokens("ligands_train")
create_file_with_space_separated_string_of_tokens("proteins_train")
create_file_with_space_separated_string_of_tokens("ligands_test")
create_file_with_space_separated_string_of_tokens("proteins_test")

**Checks**

In [0]:
#!diff /content/ligands /content/ligands1

In [0]:
for file_name in ['proteins_train', 'proteins_test']:
  file = '/content/'+file_name
  i=0
  with open(file,"r") as f:
    for line in f:
      if any(c.islower() for c in line):
        print(file,i)
      i=i+1
              



In [0]:

for file_name in ['proteins_train', 'proteins_test', 'ligands_train', 'ligands_test', 'proteins_and_ligands']:
  file = '/content/'+file_name
  i=0
  with open(file,"r") as f:  
    print(file,'number of unique items',len(set(f)))
  with open(file,"r") as f:
    for line in f:
      i=i+1  
    print(file, 'total obs', i)
  


In [0]:
prot_1_chain['Target Source Organism According to Curator or DataSource'].value_counts()

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline


t=prot_1_chain.loc[prot_1_chain['IC50_floats_data']>100]
print('IC50 greater then 100 ',t.shape)

prot_1_chain['IC50_floats_data'].plot.hist(xlim=(0,200),bins=100) 






t=prot_1_chain.loc[prot_1_chain['Kd_floats_data']>100]
print('Kd greater then 100 ',t.shape)

prot_1_chain['Kd_floats_data'].plot.hist(xlim=(0,200),bins=1000) 





prot_1_chain['Target Source Organism According to Curator or DataSource'].value_counts().plot(kind='bar', subplots=True,figsize=(20, 7))

In [0]:
columns = data.columns
for col in columns:
    print(col)

In [0]:
'''prot='MLLVSTVSAVPGLFSPPSSPSNSSQEELLDDRDPLLVRAELALLSTIFVAVALSNGLVLGALIRRGRRGRWAPMHVFISHLCLADLAVALFQVLPQLAWDATDRFHGPDALCRAVKYLQMVGMYASSYMILAMTLDRHRAICRPMLAYRHGGGARWNRPVLVAWAFSLLLSLPQLFIFAQRDVGNGSGVFDCWARFAEPWGLRAYVTWIALMVFVAPALGIAACQVLIFREIHASLVPGPSERAGRRRRGRRTGSPSEGAHVSAAMAKTVRMTLVIVIVYVLCWAPFFLVQLWAAWDPEAPLERPPFVLLMLLASLNSCTNPWIYASFSSSVSSELRSLLCCAQRHTTHSLGPQDESCATASSSLMKDTPS'

def transform(val):
  return Chem.MolToSmiles(Chem.MolFromSmiles(str(val)))

with open('/content/test',"w") as t:
  new_ds = prot_1_chain.loc[prot_1_chain['BindingDB Target Chain  Sequence'] == prot]
  print(new_ds.shape)
  new_ds['new_lig'] = new_ds['Ligand SMILES'].apply(transform)
  new_ds[['BindingDB Target Chain  Sequence','Ligand SMILES','new_lig']].to_csv(t,sep='\t')'''
  

In [0]:
prot='MRLPGAMPALALKGELLLLSLLLLLEPQISQGLVVTPPGPELVLNVSSTFVLTCSGSAPVVWERMSQEPPQEMAKAQDGTFSSVLTLTNLTGLDTGEYFCTHNDSRGLETDERKRLYIFVPDPTVGFLPNDAEELFIFLTEITEITIPCRVTDPQLVVTLHEKKGDVALPVPYDHQRGFSGIFEDRSYICKTTIGDREVDSDAYYVYRLQVSSINVSVNAVQTVVRQGENITLMCIVIGNEVVNFEWTYPRKESGRLVEPVTDFLLDMPYHIRSILHIPSAELEDSGTYTCNVTESVNDHQDEKAINITVVESGYVRLLGEVGTLQFAELHRSRTLQVVFEAYPPPTVLWFKDNRTLGDSSAGEIALSTRNVSETRYVSELTLVRVKVAEAGHYTMRAFHEDAEVQLSFQLQINVPVRVLELSESHPDSGEQTVRCRGRGMPQPNIIWSACRDLKRCPRELPPTLLGNSSEEESQLETNVTYWEEEQEFEVVSTLRLQHVDRPLSVRCTLRNAVGQDTQEVIVVPHSLPFKVVVISAILALVVLTIISLIILIMLWQKKPRYEIRWKVIESVSSDGHEYIYVDPMQLPYDSTWELPRDQLVLGRTLGSGAFGQVVEATAHGLSHSQATMKVAVKMLKSTARSSEKQALMSELKIMSHLGPHLNVVNLLGACTKGGPIYIITEYCRYGDLVDYLHRNKHTFLQHHSDKRRPPSAELYSNALPVGLPLPSHVSLTGESDGGYMDMSKDESVDYVPMLDMKGDVKYADIESSNYMAPYDNYVPSAPERTCRATLINESPVLSYMDLVGFSYQVANGMEFLASKNCVHRDLAARNVLICEGKLVKICDFGLARDIMRDSNYISKGSTFLPLKWMAPESIFNSLYTTLSDVWSFGILLWEIFTLGGTPYPELPMNEQFYNAIKRGYRMAQPAHASDEIYEIMQKCWEEKFEIRPPFSQLVLLLERLLGEGYKKKYQQVDEEFLRSDHPAILRSQARLPGFHGLRSPLDTSSVLYTAVQPNEGDNDYIIPLPDPKPEVADEGPLEGSPSLASSTLNEVNTSSTISCDSPLEPQDEPEPEPQLELQVEPEPELEQLPDSGCPAPRAEAEDSFL'
new_ds = prot_1_chain.loc[prot_1_chain['BindingDB Target Chain  Sequence'] == prot]

with open('/content/prot',"w") as file:
  new_ds.to_csv(file,sep='\t')

In [0]:
new_ds.shape

In [0]:
prot_1_chain['BindingDB Target Chain  Sequence'].value_counts()

In [0]:
ids = list(set(list(prot_1_chain['Ligand SMILES'])))
print(len(ids))